In [ ]:
from quantrion import settings

settings.DEFAULT_TIMEFRAME = "5min"

from quantrion.asset.file import CSVUSStock

import asyncio
import os
import multiprocessing
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2
print(f"{multiprocessing.cpu_count()} CPUs")

In [ ]:
stock = CSVUSStock("AAPL", "files/stocks/AAPL.csv")

In [ ]:
start = stock.localize(pd.Timestamp.utcnow()) - pd.Timedelta("5000d")
stock.bars._curr_idx = stock.bars._df.shape[0] - 1
data = await stock.bars.get(start)
data.head()

In [ ]:
start_train, end_train = data.index[0], data.index[int(data.shape[0] * 0.8)]
start_test, end_test = end_train + pd.Timedelta("5min"), data.index[-1]
data_train = data[:end_train]
data_test = data[start_test:]

In [ ]:
from enum import auto

start, end = stock.localize(pd.Timestamp("2022-08-03")), stock.localize(
    pd.Timestamp("2022-08-18")
)
freq = "15min"

df = await stock.bars.get(start, end, freq=freq)
df = stock.restriction.filter(df)
df["atr"] = await stock.bars.get_atr(start, end, freq=freq)
df[["supertrend", "bullish"]] = await stock.bars.get_supertrend(
    start, end, freq, n=10, k=3
)
df[["long_supertrend", "long_bullish"]] = await stock.bars.get_supertrend(
    start, end, freq, n=40, k=5
)
df = stock.restriction.filter(df)


def plot_data(df):
    supertrend, bullish = df["supertrend"], df["bullish"].astype(bool)
    long_supertrend, long_bullish = df["long_supertrend"], df["long_bullish"].astype(
        bool
    )
    layout = go.Layout(
        autosize=False,
        width=1200,
        height=700,
    )
    fig = go.Figure(layout=layout)
    fig = make_subplots(specs=[[{"secondary_y": True}]], figure=fig)
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
            name="Candles",
        ),
    )
    lower_supertrend = supertrend[~bullish].reindex(supertrend.index)
    upper_supertrend = supertrend[bullish].reindex(supertrend.index)
    long_lower_supertrend = long_supertrend[~long_bullish].reindex(
        long_supertrend.index
    )
    long_upper_supertrend = long_supertrend[long_bullish].reindex(long_supertrend.index)
    fig.add_trace(
        go.Scatter(
            x=lower_supertrend.index,
            y=lower_supertrend,
            name="Supertrend",
            marker={"color": "#F00"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=upper_supertrend.index,
            y=upper_supertrend,
            name="Supertrend",
            marker={"color": "#0F0"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=long_lower_supertrend.index,
            y=long_lower_supertrend,
            name="Long Supertrend",
            marker={"color": "#F00"},
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=long_upper_supertrend.index,
            y=long_upper_supertrend,
            name="Long Supertrend",
            marker={"color": "#0F0"},
        ),
    )
    if "positions" in df.columns:
        positions = df["positions"]
        fig.add_trace(
            go.Scatter(x=positions.index, y=positions, name="Positions"),
            secondary_y=True,
        )
    if "strategy" in df.columns:
        layout = go.Layout(
            autosize=False,
            width=1200,
            height=700,
        )
        fig2 = go.Figure(layout=layout)
        fig2 = make_subplots(specs=[[{"secondary_y": True}]], figure=fig2)
        fig2.add_traces(
            [
                go.Scatter(x=df.index, y=df["strategy"], name="strategy"),
                go.Scatter(x=df.index, y=df["benchmark"], name="benchmark"),
                go.Scatter(x=df.index, y=df["trades"], name="trades"),
            ],
            secondary_ys=[False, False, True],
        )
        fig2.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                dict(
                    bounds=["sat", "mon"]
                ),  # hide weekends, eg. hide sat to before mon
                dict(
                    bounds=[16, 9.5], pattern="hour"
                ),  # hide hours outside of 9.30am-4pm
            ],
        )
        fig2.show()

    # fig.add_trace(
    #     go.Scatter(x=df["atr"].index, y=df["atr"], name="ATR"),
    #     secondary_y=True
    # )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
            dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
        ],
    )
    fig.show()

In [ ]:
plot_data(df)

In [ ]:
from datetime import time


spread = 0.012 / 2


async def process_bars(
    stock: CSVUSStock,
    start: pd.Timestamp,
    end: pd.Timestamp,
    freq: str = "5min",
    short_n=10,
    long_n=40,
    short_k=3,
    long_k=5,
):
    stock.bars._curr_idx = stock.bars._df.shape[0] - 1
    bars = await stock.bars.get(start, end, freq)
    atr_ser = await stock.bars.get_atr(start, end, freq)
    supertrend = await stock.bars.get_supertrend(
        bars.index[0], bars.index[-1], freq, n=short_n, k=short_k
    )
    long_supertrend = await stock.bars.get_supertrend(
        bars.index[0], bars.index[-1], freq, n=long_n, k=long_k
    )
    prev_bar, prev_atr, prev_supertrend, prev_long_supertrend = (
        bars.iloc[0],
        atr_ser.iloc[0],
        supertrend.iloc[0],
        long_supertrend.iloc[0],
    )
    if stock.restriction is not None:
        bars = stock.restriction.filter(bars)
    bars["log_volume"] = np.log(bars["volume"] + 0.01)
    bars["quantile"] = (
        bars["log_volume"].rolling(long_n).mean()
        + 1.5 * bars["log_volume"].rolling(long_n).std()
    )
    supertrend = stock.restriction.filter(supertrend)
    long_supertrend = stock.restriction.filter(long_supertrend)
    bars[["supertrend", "bullish"]] = supertrend
    bars[["long_supertrend", "long_bullish"]] = long_supertrend
    bars = bars.dropna()
    bars["trend_change"] = bars["bullish"].diff().fillna(0)
    bars["long_trend_change"] = bars["long_bullish"].diff().fillna(0)
    bars.loc[bars["long_trend_change"] != 0, "positions"] = 0
    long_bullish = bars["long_bullish"].astype(bool)
    high_volume = bars["log_volume"] > bars["quantile"]
    bars.loc[(bars["long_trend_change"] == 1), "positions"] = 0
    bars.loc[(bars["trend_change"] == 1) & long_bullish & high_volume, "positions"] = 1
    bars.loc[
        (bars["trend_change"] == -1) & ~long_bullish & high_volume, "positions"
    ] = -1
    # bars.loc[(bars["trend_change"] == -1) & long_bullish, "positions"] = 0
    # bars.loc[bars.index.time > time.fromisoformat("15:30"),"positions"] = 0
    bars["positions"] = bars["positions"].ffill().shift(1).fillna(0)
    bars.loc[bars.index.time > time.fromisoformat("15:50"), "positions"] = 0
    bars["trades"] = bars["positions"].diff().abs().fillna(0).cumsum()
    bars["price_change"] = np.log(bars["close"] / bars["close"].shift(1)).fillna(0)
    bars["profit"] = bars["price_change"] * bars["positions"]
    bars["strategy"] = (
        np.exp(bars["profit"].cumsum()) - bars["trades"] * spread / bars["close"]
    )
    bars["benchmark"] = np.exp(bars["price_change"].cumsum()) - spread / bars["close"]
    cummax = bars["strategy"].cummax()
    rev_cummin = bars["strategy"].iloc[::-1].cummin().iloc[::-1]
    bars["drawdown"] = cummax - rev_cummin
    return bars

In [ ]:
symbol = "AAPL"
(
    await process_bars(
        CSVUSStock(symbol, f"files/stocks/{symbol}.csv"), start_train, end_train
    )
).head(10)

In [ ]:
import optuna
from threading import Thread
from time import sleep
import multiprocess


def run_for(symbol):
    loop = asyncio.new_event_loop()
    stock = CSVUSStock(symbol, f"files/stocks/{symbol}.csv")
    data = stock.bars._df
    start_train, end_train = data.index[0], data.index[int(data.shape[0] * 0.8)]

    def objective(trial):
        short_n = trial.suggest_int("short_n", 10, 40)
        long_n = trial.suggest_int("long_n", 40, 200)
        short_k = trial.suggest_float("short_k", 0.5, 3)
        long_k = trial.suggest_float("long_k", 3, 8)

        df = loop.run_until_complete(
            process_bars(
                stock,
                start_train,
                end_train,
                freq="15min",
                short_n=short_n,
                long_n=long_n,
                short_k=short_k,
                long_k=long_k,
            ),
        )
        n_trades = df["trades"][-1]
        if n_trades < 10:
            return 0
        hit_ratio = (df["profit"] > 0).sum() / (df["profit"] != 0).sum()
        # return (df["strategy"][-1] - df["strategy"][0]) / df["strategy"][0]
        # return hit_ratio
        return (df["strategy"].iloc[-1] - df["strategy"].iloc[0]) / df["drawdown"].max()

    study = optuna.create_study(direction="maximize")
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=100)
    result = study.trials_dataframe().sort_values("value")
    result["symbol"] = symbol
    print(f"Done for {symbol}. Best score {result.iloc[-1]['value']:2f}")
    return result


symbols = sorted(
    [
        file.replace(".csv", "")
        for file in os.listdir("files/stocks")
        if file.endswith(".csv")
    ]
)
n_cpus = multiprocessing.cpu_count()
with multiprocess.Pool(n_cpus) as pool:
    study_df = pool.map(run_for, symbols)
study_df = pd.concat(study_df)

In [ ]:
study_df.to_csv("files/supertrend_study.csv", index=False)

In [ ]:
from random import random

symbol = symbols[int(random() * len(symbols))]
print(symbol)
stock = CSVUSStock(symbol, f"files/stocks/{symbol}.csv")
df = await process_bars(
    # stock, start_test, end_test, freq="15min", short_n=7, long_n=170, short_k=2, long_k=9
    stock,
    start_test,
    end_test,
    freq="15min",
    short_n=15,
    long_n=40,
    short_k=1.3,
    long_k=3.6,
)
n_items = len(df)
s = random() * 0.8
e = s + 0.2 * random()
start, end = df.index[int(n_items * s)], df.index[int(n_items * e)]
# print(df[start:end])
plot_data(df)

In [ ]:
study_df.sort_values("value", ascending=False).head()

In [ ]:
study_df["short_n_group"] = (study_df["params_short_n"] / 2).astype(int) * 2
study_df["long_n_group"] = (study_df["params_long_n"] / 10).astype(int) * 10
study_df["short_k_group"] = (study_df["params_short_k"] / 0.1).astype(int) * 0.1
study_df["long_k_group"] = (study_df["params_long_k"] / 0.2).astype(int) * 0.2

In [ ]:
study_df.head()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

all_cols = ["short_n_group", "long_n_group", "short_k_group", "long_k_group", "value"]
cols = ["short_k_group", "long_k_group"]
filter_groups = {
    "short_n_group": [12],
    "long_n_group": [40],
    # "short_k_group": [1.4, 1.5, 1.6],
    # "long_k_group": [4.6, 4.7, 4.8],
}
heatmap_df = study_df[all_cols].copy()
for col, groups in filter_groups.items():
    heatmap_df = heatmap_df[heatmap_df[col].isin(groups)]
heatmap_df = heatmap_df[["value", *cols]]
heatmap_df = heatmap_df.groupby(cols).mean().reset_index()
heatmap_df = heatmap_df.pivot_table(index=cols[0], columns=cols[1], values="value")

layout = go.Layout(height=800, width=800)
fig = go.Figure(layout=layout)
fig = make_subplots(figure=fig, rows=2, cols=2)
fig.add_trace(
    go.Heatmap(
        x=heatmap_df.columns,
        y=heatmap_df.index,
        z=heatmap_df.values,
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=heatmap_df.index,
        x=heatmap_df.mean(axis=1).values,
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=heatmap_df.columns,
        y=heatmap_df.mean(axis=0).values,
    ),
    row=2,
    col=1,
)
fig.show()